In [2]:
import pandas

In [4]:
drug = pandas.read_csv("drug.csv")
drug

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,NaN,drugX
4,61,F,LOW,HIGH,18.043,drugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NaN,14.020,drugX


In [5]:
# Check for missing values

missing_values = drug.isnull().sum()
print("Missing Values:")
print(missing_values)

Missing Values:
Age            0
Sex            0
BP             2
Cholesterol    2
Na_to_K        1
Drug           0
dtype: int64


In [6]:
# remove missing values
drug.dropna(inplace=True)
drug

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
4,61,F,LOW,HIGH,18.043,drugY
5,22,F,NORMAL,HIGH,8.607,drugX
...,...,...,...,...,...,...
194,46,F,HIGH,HIGH,34.686,drugY
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX


In [7]:
# Encode categorical variable
from sklearn.preprocessing import LabelEncoder

categorical_features = ["Sex", "BP", "Cholesterol", "Drug"]
for feature in categorical_features:
    le = LabelEncoder()
    drug[feature] = le.fit_transform(drug[feature])

print("<--- Data after remove missing values --->")
drug


<--- Data after remove missing values --->


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,0,0,0,25.355,4
1,47,1,1,0,13.093,2
2,47,1,1,0,10.114,2
4,61,0,1,0,18.043,4
5,22,0,2,0,8.607,3
...,...,...,...,...,...,...
194,46,0,0,0,34.686,4
195,56,0,1,0,11.567,2
196,16,1,1,0,12.006,2
197,52,1,2,0,9.894,3
